# Here, I will analyze the saved study

In [ ]:
import joblib
import numpy as np
import tensorflow as tf

from tensorflow import keras
from keras import layers
import optuna

import matplotlib.pyplot as plt

### Setting Up GPU as a training device

In [ ]:
physical_devices = tf.config.list_physical_devices()

print(physical_devices)

In [ ]:
tf.config.set_visible_devices(physical_devices[1], 'GPU')

### Loading Data

In [ ]:
N_TRAIN_EXAMPLES = 50000
N_TEST_EXAMPLES = 10000

CLASSES=10

In [ ]:
(X_train, Y_train), (X_test, Y_test) = keras.datasets.cifar10.load_data()

import numpy as np
from sklearn.model_selection import train_test_split

# Concatenate train and test images
X = np.concatenate((X_train, X_test))
y = np.concatenate((Y_train, Y_test))

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=N_TRAIN_EXAMPLES, test_size=N_TEST_EXAMPLES, random_state=1)

# Getting dummy variables
y_train_fixed = np.zeros((y_train.shape[0], 10))
y_test_fixed = np.zeros((y_test.shape[0], 10))

i = 0
for [val] in y_train:
  y_train_fixed[i][val] = 1
  i += 1
  
i = 0
for [val] in y_test:
  y_test_fixed[i][val] = 1
  i += 1

### Loading study

In [ ]:
study = joblib.load('studies/optuna_study_saving_test.pkl')

print(study.best_params)
print(study.best_value)

### Building model

In [ ]:
model = keras.Sequential(
  [
    keras.Input(shape=(32, 32, 3)),
    layers.Conv2D(64, kernel_size=3, padding='same', activation="relu"),
    layers.Dropout(0.1),
    layers.Conv2D(64, kernel_size=3, strides=1, padding='same', activation="relu"),
    layers.Dropout(0.1),
    layers.Conv2D(64, kernel_size=5, strides=2, padding='same', activation="relu"),
    layers.Dropout(0.2),
    layers.MaxPooling2D(pool_size=3),
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(CLASSES, activation="softmax"),
  ]
)

model.summary()

In [ ]:
optimizer = keras.optimizers.Adam()
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
%%time
batch_size = 256
epochs = 20
validation_split = 0.2
history = model.fit(X_train, y_train_fixed, batch_size=batch_size, epochs=epochs, validation_split=validation_split)

### Save model to file

In [ ]:
model.save("models/cifar10_model_1.keras")

In [ ]:
# Plotting the loss curve
plt.figure(figsize=[6, 4])
plt.plot(history.history['loss'], 'black', linewidth=2.0)
plt.plot(history.history['val_loss'], 'blue', linewidth=2.0)
plt.legend(['Training Loss', 'Validation Loss'], fontsize=12)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Loss', fontsize=10)
plt.title('Loss Curves', fontsize=12)

In [ ]:
# Plotting the accuracy curve
plt.figure(figsize=[6, 4])
plt.plot(history.history['accuracy'], 'black', linewidth=2.0)
plt.plot(history.history['val_accuracy'], 'blue', linewidth=2.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=12)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Accuracy', fontsize=10)
plt.title('Accuracy Curves', fontsize=12)